<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><span><a href="#Импорты" data-toc-modified-id="Импорты-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Импорты</a></span></li><li><span><a href="#Изучение-данных" data-toc-modified-id="Изучение-данных-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Изучение данных</a></span></li><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Подготовка данных</a></span></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#RandomForest" data-toc-modified-id="RandomForest-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>RandomForest</a></span></li><li><span><a href="#Логическая-регрессия" data-toc-modified-id="Логическая-регрессия-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Логическая регрессия</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проекта" data-toc-modified-id="Чек-лист-проекта-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек лист проекта</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 


Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

### Импорты

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from sklearn.feature_extraction.text import TfidfVectorizer
from pymystem3 import Mystem
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.metrics import f1_score

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

STATE = 12345 # константа рандома

In [2]:
tweets = pd.read_csv('/datasets/toxic_comments.csv')

### Изучение данных

In [3]:
# функция для информативного предоставления данных
def data_info(df):
        display(df.head(5))
        print(50*'-')
        print("Дубликатов в данных: ", df.duplicated().sum())
        print(50*'-')
        df.info()
        print(50*'-')
        display(df.describe())
        print(50*'-')
        print("Пропусков в данных: ")
        def data_isna(df):
            isna = pd.DataFrame(round(df.isna().mean()*100,)).style.background_gradient('coolwarm')
            return isna
        display(data_isna(df))

In [4]:
data_info(tweets)

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


--------------------------------------------------
Дубликатов в данных:  0
--------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB
--------------------------------------------------


,Unnamed: 0,toxic
count,159292.000000,159292.000000
mean,79725.697242,0.101612
std,46028.837471,0.302139
min,0.000000,0.000000
25%,39872.750000,0.000000
50%,79721.500000,0.000000
75%,119573.250000,0.000000
max,159450.000000,1.000000


--------------------------------------------------
Пропусков в данных: 


,0
Unnamed: 0,0.000000
text,0.000000
toxic,0.000000


In [5]:
display("Колиество положительных и отрицательных коментов: ", tweets['toxic'].value_counts())

'Колиество положительных и отрицательных коментов: '

0    143106
1     16186
Name: toxic, dtype: int64

In [6]:
display("Соотношение отрицательных к положительным отзывам : ", 
        tweets['toxic'].value_counts()[0] / tweets['toxic'].value_counts()[1])

'Соотношение отрицательных к положительным отзывам : '

8.841344371679229

    Наша база твитов состоит из почти 160 тысяч строк с твитами.
    Присутствует столбец "Unnamed" который на первый взгляд дублирует индексы
    Пропусков нету как и дубликатов
    Для более "ровной" работы необходимо будет привести все твиты в один вид(нижний регистр и удалить все лишнее)
    Из проблем для обучения моделей может быть только несбалансированность классов - негативных в 9 раз больше.    

### Подготовка данных

In [7]:
#Перевод в нижний регистр

tweets['text'] = tweets['text'].str.lower()

In [8]:
lemmatizer  = WordNetLemmatizer()

In [9]:
def clear_lemm_text(text):
    clean_text = []
    pattern = r'[^a-zA-Z0-9]' 
    for tweet in text:
        cleared_text = re.sub(pattern, " ", tweet)
        clean = cleared_text.split()
        clean_text.append(' '.join(lemmatizer.lemmatize(word) for word in clean))
    
    return clean_text

In [10]:
tweets['lemm_text'] = clear_lemm_text(tweets['text'])

In [11]:
tweets['text'][0]
# 16 слво vandalisms

"explanation\nwhy the edits made under my username hardcore metallica fan were reverted? they weren't vandalisms, just closure on some gas after i voted at new york dolls fac. and please don't remove the template from the talk page since i'm retired now.89.205.38.27"

In [12]:
tweets['lemm_text'][0]
# тут в тексте у него нету окончания, значит лематизаци прошла успешно, если это можно так назвать...

'explanation why the edits made under my username hardcore metallica fan were reverted they weren t vandalism just closure on some gas after i voted at new york doll fac and please don t remove the template from the talk page since i m retired now 89 205 38 27'

Данные готовы к обучению моделей

## Обучение

In [13]:
features = tweets.drop(['toxic', 'Unnamed: 0', 'text'] , axis=1)
target = tweets['toxic']

In [14]:
print(features.shape)
print(target.shape)

(159292, 1)
(159292,)


In [15]:
features_train, features_valid_test, target_train, target_valid_test = train_test_split(features,
                                                    target,
                                                    train_size=0.6,
                                                    random_state=STATE, 
                                                    stratify=target)
# выделил 50% данных на тренировочную выборку, остальные данные разделю поровну между валидационнрй и тестовой

In [16]:
features_valid, features_test, target_valid, target_test = train_test_split(features_valid_test,
                                                    target_valid_test,
                                                    train_size=0.5,
                                                    random_state=STATE, 
                                                    stratify=target_valid_test)

In [17]:
#векторизация данных
nltk.download('stopwords')
stopwords = set(stopwords.words('english'))

count_tf_idf = TfidfVectorizer(stop_words=stopwords)

features_train = count_tf_idf.fit_transform(features_train['lemm_text'].values)
features_valid = count_tf_idf.transform(features_valid['lemm_text'].values)
features_test = count_tf_idf.transform(features_test['lemm_text'].values)

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### RandomForest

In [18]:
f1_tabl=[]

In [19]:
%%time
param_grid = {'max_depth': range(2, 11, 2),
              'n_estimators': range(10, 50, 10)
              }             

model = RandomForestClassifier(class_weight = 'balanced', random_state=STATE)
# так как данные не сбалансированы воспользуемся параметром  class_weight = 'balanced'
search = GridSearchCV(model, param_grid, cv=3, scoring='f1')
search.fit(features_train, target_train)
print('Лучшее значение F1 для случайного леса ={}'
      '\n при значениях гиперпараметров: {}'.format(search.best_score_,search.best_params_))

Лучшее значение F1 для случайного леса =0.3450242801546892
 при значениях гиперпараметров: {'max_depth': 10, 'n_estimators': 40}
CPU times: user 1min 43s, sys: 1.32 s, total: 1min 45s
Wall time: 1min 45s


In [20]:
f1_tabl.append(search.best_score_)

### Логическая регрессия

In [21]:
%%time
model = LogisticRegression(class_weight = 'balanced')
# тоже, так как данные не сбалансированы воспользуемся параметром  class_weight = 'balanced'
model.fit(features_train,target_train)
pred_valid = model.predict(features_valid)

CPU times: user 12.9 s, sys: 27.5 s, total: 40.4 s
Wall time: 40.5 s


/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [22]:
f_1 = f1_score(target_valid, pred_valid)
print('F1 для Логической регресии =', f_1)

F1 для Логической регресии = 0.7521250342747464


In [23]:
f1_tabl.append(f_1)

In [24]:
%%time
model = LogisticRegression()
# тоже, так как данные не сбалансированы воспользуемся параметром  class_weight = 'balanced'
model.fit(features_train,target_train)
pred_valid_db = model.predict(features_valid)
f_1_disbal = f1_score(target_valid, pred_valid_db)
print('F1 для Логической регресии =', f_1_disbal)

F1 для Логической регресии = 0.7257207890743551
CPU times: user 12.4 s, sys: 26.2 s, total: 38.6 s
Wall time: 38.7 s


/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [25]:
f1_tabl.append(f_1_disbal)

In [26]:
f1_mera=pd.DataFrame(f1_tabl, index=['RF', 'LogReg', 'LR_disbalans'], columns=['f1'])

In [27]:
display(f1_mera)

,f1
RF,0.345024
LogReg,0.752125
LR_disbalans,0.725721


Исходя из вычислений и последней таблицы можно сдедать вывод что для поиска токсичных твитов модель случайного леса полностью не подходит, так как при сбалансированной метрики, показатель F1 всего `0,345`.
    
Тогда как Логическая регрессия показывает неплохой результат F1 1 в `0,72`, на несбалансированных класах. 

А уже при указании `class_weight = 'balanced' ` F1 поднялось до `0,75`, что для нас является хорошим показателем.

Отсюда следует, что Логическая регрессия с `class_weight = 'balanced'` - идеальный вариант для поиска токсичных коментариев.

## Выводы

In [28]:
pred_test = model.predict(features_test)
f_1_test = f1_score(target_test, pred_test)
print('F1 на тестовой выборке:', f_1)

F1 на тестовой выборке: 0.7521250342747464


**Вывод** 

В данном проекте мне пришлось обрабатывать твиты людей и обучение модели на поиск токсичных твитов, очистив текст от лишних символов, разделив его и проведя лемитизацию мне удалось добиться показателя f1 на тестовой выборке = `0.752`.
В процессе подготовки данных выявлено что классы данных не сбалансированны, и было проверенно обучение моделей на сбалансированных и не сбалансированных данных. 
Так как мне удалось добиться метрики не противоречащей требованию, можно считать выбранную модель - `LogisticRegression(class_weight = 'balanced')` - верным решением для выполнения данной задачи